In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install dlib opencv-python-headless
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

--2025-09-07 09:07:30--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-09-07 09:07:30--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  20.1MB/s    in 3.0s    

2025-09-07 09:07:34 (20.1 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.


In [ ]:
!pip install mediapipe opencv-python-headless tqdm

In [ ]:
import dlib
import cv2
import os

In [ ]:


# input and output paths
input_video_dir = '/content/drive/MyDrive/Actor_01'
output_image_dir = '/content/drive/My Drive/RAVDESS_frames'
os.makedirs(output_image_dir, exist_ok=True)

# # define emotion mapping
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# iterate through video files
for video_file in os.listdir(input_video_dir):
    video_path = os.path.join(input_video_dir, video_file)

    # get emotion code (filename format: "01-01-03-01.mp4")
    emotion_code = video_file.split("-")[2]
    emotion = emotion_map.get(emotion_code, "unknown")


    emotion_dir = os.path.join(output_image_dir, emotion)
    os.makedirs(emotion_dir, exist_ok=True)


    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video: {video_file}")
        continue

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # extract one frame every 30 frames
        if frame_count % 30 == 0:
            frame_name = f"{os.path.splitext(video_file)[0]}_frame{frame_count}.jpg"
            save_path = os.path.join(emotion_dir, frame_name)
            cv2.imwrite(save_path, frame)

        frame_count += 1

    cap.release()
    print(f"Extracted frames from video: {video_file}")

print("Frame extraction completed!")

Extracted frames from video: 01-02-01-01-01-01-12.mp4
Extracted frames from video: 02-02-02-02-02-01-01.mp4
Extracted frames from video: 02-02-03-01-01-02-01.mp4
Extracted frames from video: 02-02-03-01-02-01-01.mp4
Extracted frames from video: 02-02-03-01-02-02-01.mp4
Extracted frames from video: 02-02-03-02-01-01-01.mp4
Extracted frames from video: 02-02-03-02-01-02-01.mp4
Extracted frames from video: 02-02-04-01-01-01-01.mp4
Extracted frames from video: 02-02-03-02-02-01-01.mp4
Extracted frames from video: 02-02-03-02-02-02-01.mp4
Extracted frames from video: 02-02-04-01-01-02-01.mp4
Extracted frames from video: 02-02-04-01-02-01-01.mp4
Extracted frames from video: 02-02-04-01-02-02-01.mp4
Extracted frames from video: 02-02-04-02-01-01-01.mp4
Extracted frames from video: 02-02-04-02-02-01-01.mp4
Extracted frames from video: 02-02-04-02-01-02-01.mp4
Extracted frames from video: 02-02-04-02-02-02-01.mp4
Extracted frames from video: 02-02-05-01-01-01-01.mp4
Extracted frames from video:

In [ ]:
import os, cv2, numpy as np
from glob import glob
from tqdm import tqdm
import mediapipe as mp

# ====== Path settings (edit here) ======
SRC_SPLIT_ROOT = "/content/drive/MyDrive/RAVDESS_split_b"     # input root
DST_SPLIT_ROOT = "/content/drive/MyDrive/Ravdata_Ori_split"    # output root
OUT_SIZE = 224

os.makedirs(DST_SPLIT_ROOT, exist_ok=True)

mp_face = mp.solutions.face_mesh

#landmark indices
FACE_IDX = {
    "left_eye":  [33, 133, 160, 159, 158, 144, 153, 154, 155],
    "right_eye": [362, 263, 387, 386, 385, 373, 380, 381, 382],
    "nose":      [1, 2, 98, 327, 168, 195, 5, 4],
    "mouth":     [78, 308, 13, 14, 17, 82, 87, 312, 317],
}

def crop_by_landmarks(img, landmarks, idxs, margin=0.20):
    # crop face part by landmarks
    h, w = img.shape[:2]
    pts = np.array([(int(lm.x*w), int(lm.y*h))
                    for i, lm in enumerate(landmarks) if i in idxs])
    if len(pts) == 0: return None
    x1,y1 = pts.min(0); x2,y2 = pts.max(0)
    mx = int((x2-x1)*margin); my = int((y2-y1)*margin)
    x1 = max(0, x1-mx); y1 = max(0, y1-my)
    x2 = min(w, x2+mx); y2 = min(h, y2+my)
    crop = img[y1:y2, x1:x2]
    return crop if crop.size else None

def make_grid2x2(a,b,c,d, out_size=224):
    # create 2x2 grid image
    def norm(s):
        if s is None or s.size == 0:
            return np.zeros((out_size//2, out_size//2, 3), dtype=np.uint8)
        return cv2.resize(s, (out_size//2, out_size//2))
    A,B,C,D = map(norm, [a,b,c,d])
    top = np.hstack([A,B]); bot = np.hstack([C,D])
    return np.vstack([top, bot])

def process_one(img_path, face_mesh):
    # process one image
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None: return None
    res = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not res.multi_face_landmarks: return None
    lms = res.multi_face_landmarks[0].landmark
    le = crop_by_landmarks(img, lms, FACE_IDX["left_eye"])
    re = crop_by_landmarks(img, lms, FACE_IDX["right_eye"])
    no = crop_by_landmarks(img, lms, FACE_IDX["nose"])
    mo = crop_by_landmarks(img, lms, FACE_IDX["mouth"])
    return make_grid2x2(le, re, no, mo, OUT_SIZE)

splits = ["train", "validation", "test"]
with mp_face.FaceMesh(static_image_mode=True, refine_landmarks=True, max_num_faces=1) as face_mesh:
    for sp in splits:
        for cls in os.listdir(os.path.join(SRC_SPLIT_ROOT, sp)):
            src_dir = os.path.join(SRC_SPLIT_ROOT, sp, cls)
            if not os.path.isdir(src_dir): continue
            dst_dir = os.path.join(DST_SPLIT_ROOT, sp, cls)
            os.makedirs(dst_dir, exist_ok=True)

            imgs = [p for p in glob(os.path.join(src_dir, "*"))
                    if p.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]

            ok = 0
            for p in tqdm(imgs, desc=f"{sp}/{cls}"):
                out = process_one(p, face_mesh)
                if out is None: continue
                fname = os.path.splitext(os.path.basename(p))[0] + "_grid.jpg"
                cv2.imwrite(os.path.join(dst_dir, fname), out)
                ok += 1
            print(f"[{sp}/{cls}] done: {ok}/{len(imgs)}")
print("ALL DONE ->", DST_SPLIT_ROOT)



train/neutral: 100%|██████████| 481/481 [00:33<00:00, 14.22it/s]


[train/neutral] done: 481/481


train/happy: 100%|██████████| 962/962 [01:11<00:00, 13.38it/s]


[train/happy] done: 962/962


train/calm: 100%|██████████| 1049/1049 [01:20<00:00, 13.03it/s]


...

[test/angry] done: 260/260


test/sad: 100%|██████████| 282/282 [00:18<00:00, 15.51it/s]


[test/sad] done: 282/282


test/fearful: 100%|██████████| 247/247 [00:14<00:00, 16.66it/s]


[test/fearful] done: 247/247
ALL DONE -> /content/drive/MyDrive/Ravdata_Ori_split


In [ ]:
import os, cv2, numpy as np
from glob import glob
from tqdm import tqdm
import mediapipe as mp

SRC_SPLIT_ROOT = "/content/drive/MyDrive/RAVDESS_split_b"          # input root
DST_SPLIT_ROOT = "/content/drive/MyDrive/Ravdata_face_onlyOri_split"     # output root
OUT_SIZE = 224
MARGIN = 0.15

os.makedirs(DST_SPLIT_ROOT, exist_ok=True)
mp_det = mp.solutions.face_detection

def expand_box(x1,y1,x2,y2,w,h,m=0.15):
    # expand the bounding box by margin
    bw, bh = x2-x1, y2-y1
    x1 = max(0, int(x1 - bw*m)); y1 = max(0, int(y1 - bh*m))
    x2 = min(w, int(x2 + bw*m)); y2 = min(h, int(y2 + bh*m))
    return x1,y1,x2,y2

def process_one(img_path, det):
    # read and detect face
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None: return None
    h,w = img.shape[:2]
    res = det.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if not res.detections: return None

    # get bounding box of face
    bb = res.detections[0].location_data.relative_bounding_box
    x1 = int(bb.xmin * w); y1 = int(bb.ymin * h)
    x2 = int((bb.xmin + bb.width) * w); y2 = int((bb.ymin + bb.height) * h)

    # expand and crop face
    x1,y1,x2,y2 = expand_box(x1,y1,x2,y2,w,h,MARGIN)
    crop = img[y1:y2, x1:x2]
    if crop.size == 0: return None
    return cv2.resize(crop, (OUT_SIZE, OUT_SIZE))

splits = ["train", "validation", "test"]
with mp_det.FaceDetection(model_selection=1, min_detection_confidence=0.5) as det:
    for sp in splits:
        for cls in os.listdir(os.path.join(SRC_SPLIT_ROOT, sp)):
            src_dir = os.path.join(SRC_SPLIT_ROOT, sp, cls)
            if not os.path.isdir(src_dir): continue
            dst_dir = os.path.join(DST_SPLIT_ROOT, sp, cls)
            os.makedirs(dst_dir, exist_ok=True)

            imgs = [p for p in glob(os.path.join(src_dir, "*"))
                    if p.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]

            ok = 0
            for p in tqdm(imgs, desc=f"{sp}/{cls}"):
                out = process_one(p, det)
                if out is None: continue
                fname = os.path.splitext(os.path.basename(p))[0] + "_face.jpg"
                cv2.imwrite(os.path.join(dst_dir, fname), out)
                ok += 1
            print(f"[{sp}/{cls}] done: {ok}/{len(imgs)}")
print("ALL DONE ->", DST_SPLIT_ROOT)

train/neutral: 100%|██████████| 481/481 [00:23<00:00, 20.30it/s]


[train/neutral] done: 481/481


train/happy: 100%|██████████| 962/962 [00:51<00:00, 18.86it/s]


[train/happy] done: 962/962


train/calm: 100%|██████████| 1049/1049 [00:56<00:00, 18.51it/s]


...

[test/angry] done: 260/260


test/sad: 100%|██████████| 282/282 [00:12<00:00, 22.05it/s]


[test/sad] done: 282/282


test/fearful: 100%|██████████| 247/247 [00:11<00:00, 20.95it/s]

[test/fearful] done: 247/247
ALL DONE -> /content/drive/MyDrive/Ravdata_face_onlyOri_split
